Replicate [Dynamic Return Dependencies Across Industries: A Machine Learning Approach](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3120110&download=yes) by David Rapach, Jack Strauss, Jun Tu and Guofu Zhou.

1) Use Keras NNs instead of linear regression

2) Add additional variables, 3 and 12-month MA, interest rate change, yield curve, Mkt-RF, seasonal dummy variables. With cross-validation and regularization we hope to do that without overfitting and possibly produce a better result.


 

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd

import time 
import datetime
import copy
import random
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from scipy.stats import chisquare

import tensorflow as tf
tf.set_random_seed(1764)
print(tf.__version__)
# confirm GPU is in use
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

import keras
from keras.layers.core import Dense, Activation
from keras.layers import Input
from keras.models import Model

from keras.layers.recurrent import LSTM, GRU
from keras.regularizers import l1
from keras.models import Sequential
from keras.models import load_model
    
import ffn
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


1.8.0
[[22. 28.]
 [49. 64.]]


Using TensorFlow backend.


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']

#for ind in industries:
#    data[ind+".3m"] = pd.rolling_mean(data[ind],3)
    
#for ind in industries:
#    data[ind+".6m"] = pd.rolling_mean(data[ind],6)

#for ind in industries:
#    data[ind+".12m"] = pd.rolling_mean(data[ind],12)
    
for ind in industries:
    data[ind+".lead"] = data[ind].shift(-1)

data = data.loc[data.index[data.index > 195911]]
data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')

nresponses = len(industries)
npredictors = data.shape[1]-nresponses

predictors = list(data.columns[:npredictors])
predictor_reverse_dict = dict([(predictors[i], i) for i in range(len(predictors))])

responses = list(data.columns[-nresponses:])
response_reverse_dict = dict([(responses[i], i) for i in range(len(responses))])

print(data.shape)

data[['Food', 'Food.lead']]


Loading data...
(697, 60)


,Food,Food.lead
yyyymm,,
195912,2.01,-4.49
196001,-4.49,3.35
196002,3.35,-1.67
196003,-1.67,1.17
196004,1.17,8.20
196005,8.20,5.39
196006,5.39,-2.11
196007,-2.11,4.57
196008,4.57,-3.88


In [3]:
# exclude 2017 and later to tie to paper
data = data.loc[data.index[data.index < 201701]]
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
data.to_csv("data.csv")
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,...,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000
mean,0.690715,0.710613,0.982321,0.701708,0.528277,0.554190,0.669460,0.650905,0.519781,0.667416,...,0.520847,0.694234,0.584175,0.511241,0.582088,0.625562,0.662219,0.702730,0.609810,0.385620
std,4.339811,5.090215,6.061582,7.180918,5.809314,4.759874,6.386027,4.928072,5.518477,7.022552,...,4.628520,6.527984,6.738979,5.055314,5.739306,5.605317,5.349341,6.104515,5.411766,5.815446
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.640000,-2.100000,-2.780000,-3.490000,-2.690000,-2.110000,-2.810000,-2.240000,-2.800000,-3.200000,...,-2.110000,-3.090000,-3.290000,-2.430000,-2.780000,-2.570000,-2.430000,-2.940000,-2.420000,-2.990000
50%,0.740000,0.710000,1.280000,0.890000,0.510000,0.750000,0.690000,0.750000,0.670000,0.630000,...,0.610000,0.970000,0.560000,0.690000,0.860000,0.940000,0.470000,1.030000,0.820000,0.470000
75%,3.120000,3.660000,4.640000,5.310000,3.720000,3.550000,4.310000,3.560000,3.760000,4.490000,...,3.360000,4.290000,4.590000,3.460000,4.060000,3.880000,4.000000,4.330000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [5]:
# annualized returns don't match Table 1, oddly
# geometric mean, annualized
pd.DataFrame((np.prod(data/100 + 1)**(12.0/len(data))-1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.074020
Beer,0.072005
Smoke,0.100147
Games,0.054031
Books,0.043953
Hshld,0.054098
Clths,0.057170
Hlth,0.065463
Chems,0.044917
Txtls,0.051888


In [6]:
# try this way, arithmetic mean then annualize (not very correct)
#print(pd.DataFrame(((desc.loc['mean']/100+1)**12-1)[:30]))
#nope

# same
pd.DataFrame(((1 + np.mean(data, axis=0)/100)**12 -1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.086108
Beer,0.088687
Smoke,0.124460
Games,0.087532
Books,0.065268
Hshld,0.068568
Clths,0.083360
Hlth,0.080966
Chems,0.064188
Txtls,0.083096


In [7]:
#annualized volatility 
pd.DataFrame((desc.loc['std']*np.sqrt(12))[:30].round(2))
# lines up with table 1

,std
Food,15.03
Beer,17.63
Smoke,21.00
Games,24.88
Books,20.12
Hshld,16.49
Clths,22.12
Hlth,17.07
Chems,19.12
Txtls,24.33


In [8]:
# skip last row to better match published r-squared
# looks like they forecast actuals 1960-2016 using 1959m12 to 2016m11
# not exact matches to Table 2 R-squared but almost within rounding error 
X = data.values[:-1,:npredictors]
Y = data.values[:-1,-nresponses:]
nrows = X.shape[0]
X.shape

(684, 30)

In [9]:
# use all predictors - higher in-sample R-squared
coef_dict_all = []
for _ in responses:
    coef_dict_all.append(range(len(predictors)))


In [10]:
# first iteration will train up to including 196911
# will use 196912 to predict 197001
# 1970101 will be first month of performance to use
# train on first 121 months up to 196912 (0:120), put first prediction in P[121] (122nd row)
# first month of performance will be 197002
FIRST_TRAIN_MONTHS = 121
FIRST_PREDICT_MONTH = FIRST_TRAIN_MONTHS # This is stupid but keeps my head straight

print(X[FIRST_TRAIN_MONTHS])
print(data.iloc[FIRST_TRAIN_MONTHS][:30])

[ -3.34  -1.95  -7.59  -7.76 -12.05  -7.5   -5.69  -7.71  -7.37  -5.26
  -9.84  -6.31  -7.15  -6.89  -9.35 -12.49  -2.34  -0.77 -12.16  -4.83
  -3.16 -11.17  -9.73  -8.89  -8.17  -8.28  -6.31 -13.12  -9.78  -6.2 ]
Food     -3.34
Beer     -1.95
Smoke    -7.59
Games    -7.76
Books   -12.05
Hshld    -7.50
Clths    -5.69
Hlth     -7.71
Chems    -7.37
Txtls    -5.26
Cnstr    -9.84
Steel    -6.31
FabPr    -7.15
ElcEq    -6.89
Autos    -9.35
Carry   -12.49
Mines    -2.34
Coal     -0.77
Oil     -12.16
Util     -4.83
Telcm    -3.16
Servs   -11.17
BusEq    -9.73
Paper    -8.89
Trans    -8.17
Whlsl    -8.28
Rtail    -6.31
Meals   -13.12
Fin      -9.78
Other    -6.20
Name: 197001, dtype: float64


In [1]:

class BacktestModel():
    
    def __init__(self, 
                 X, # predictors
                 Y, # responses
                 model=None, # model that supports fit(X,Y), predict(X) , predicts an entire row,
                 create_model=None, # or create_model which returns a model (needed for 'timestep' but slows down so pass model if dynamic not needed)
                 coef_dict_param="all", # mapping of predictors to responses ("all", "timestep", or a list of lists)
                 startindex=FIRST_TRAIN_MONTHS,
                 scaler=None,
                 fit_missing=None):
        
        self.Xrows, self.Xcols = X.shape
        self.Yrows, self.Ycols = Y.shape
        
        if self.Xrows != self.Yrows:
            raise(ValueError, "Shapes differ: X %s, Y %s" % (str(X.shape), str(Y.shape)))            
            
        self.X = X
        self.Y = Y
        self.Xscale = X.copy()
        self.Yscale = Y.copy()

        if scaler:
            # MinMaxScaler: each row (min->0, max->1) 
            # StandardScaler: each row (mean->0, SD->1)            
            # transpose, scale, transpose back because scales by columns
            print("scaler: %s " %str(scaler))
            self.Xscale = scaler().fit_transform(Xscale.transpose()).transpose()
            self.Yscale = scaler().fit_transform(Yscale.transpose()).transpose()
        
        self.model = model
        self.create_model = create_model
        self.coef_dict_param = coef_dict_param
        self.startindex = startindex
        self.fit_missing = fit_missing

    def fit_predict(self, ntrain, npredict=1, verbose=False):
        """for backtest, train model using Y v. X 
        train on first ntrain rows. if ntrain=121, fit 0:120
        predict following npredict rows 
        if npredict=1, predict row 121
        if npredict=12, predict rows 121-132
        """
        
        # fit first ntrain rows
        X_fit = self.Xscale[:ntrain]  # e.g. 0:120
        Y_fit = self.Yscale[:ntrain]
        # predict npredict rows
        X_predict = self.Xscale[ntrain:ntrain+npredict] # 121-122
        X_predict = X_predict.reshape(npredict,self.Xcols)
       
        # if no coef_dict select predictors into coef_dict
        if self.coef_dict_param == "timestep":
            msg = "Performing subset selection"
            coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'))
        # if coef_dict == "all" use all predictors for each response        
        elif self.coef_dict_param == 'all':
            msg = "Using all predictors"
            coef_dict = [range(self.Xcols) for _ in range(self.ycols)]
        else: # should check valid dict
            msg = "Using coef_dict predictors"
            coef_dict = self.coef_dict_param
        if verbose: 
            print(msg)

        if self.create_model:
            self.model = PredictWrapper(self.create_model, coef_dict, fit_missing=self.fit_missing)
            
        self.model.fit(X_fit, Y_fit, verbose=verbose)
        return self.model.predict(X_predict, verbose=verbose)

    # predict all months
    # initial train_months = 120 -> train first model on 120 rows
    # first prediction will be in P[120] (121st row)
    # step = 6 -> predict following 6 rows, then step forward 6 months at a time
    # initialize predictions matrix self.P
    
    # use either step or folds
    # step, do range(self.startindex, nrows, step)
    # folds, at each fold train 0:startfold, predict startfold+1:endfold
    # store only out-of-sample predictions in P, calc out-of-sample MSE
    
    # using a step > 1 or folds is quicker, for quicker xval, or to speed up by not estimating model at each timestep

    def gen_predictions(self,
                        step=1, 
                        splits=None,
                        verbose=False):

        self.P = np.zeros_like(self.Y)

        progress_i = 0
        self.nrows, self.ycols = Y.shape
        
        if splits:
            month_indexes = splits[:-1] # last index is nrows
        else:
            # create list of steps
            month_indexes = list(range(self.startindex, nrows, step))
        steps = [month_indexes[i+1]-month_indexes[i] for i in range(len(month_indexes)-1)]
        # last step -> end
        steps.append(self.nrows - month_indexes[-1])
        
        if verbose:
            print ("Steps: " + str(month_indexes))

        for month_index, forecast_rows in zip(month_indexes, steps):
            if verbose:
                print("Training on first %d rows (%d:%d), putting predictions in rows %s" % (month_index, 
                                                                                            0, month_index-1, 
                                                                                            str(range(month_index,month_index+forecast_rows))))
            predictions = self.fit_predict(month_index, forecast_rows, verbose=False)
            
            first_pred_row = month_index
            for row_index in range(forecast_rows):
                self.P[first_pred_row+row_index] = predictions[row_index]
            sys.stdout.write('.')
            progress_i += 1
            if progress_i % 80 == 0:
                print("")
                print("%s Still training step %d of %d" % (time.strftime("%H:%M:%S"), progress_i, len(month_indexes)))
            sys.stdout.flush()
        print("")
        

    def evaluate_predictions(self):
        
        # evaluate prediction (can move to separate function)
        msetemp = (self.P[self.startindex:]-self.Yscale[self.startindex:])**2
        #remove nans
        msetemp = msetemp[~np.isnan(msetemp)]
        mse = np.mean(msetemp)
        print("MSE across all predictions: %.4f" % mse)
        
        # force unpredicted ys to be nans, then remove nans
        vartemp = self.Yscale[self.startindex:] - self.P[self.startindex:] + self.P[self.startindex:]
        vartemp = vartemp[~np.isnan(vartemp)]
        y_variance = np.var(vartemp[self.startindex:])
        print("Variance: %.4f" % (y_variance))
        print("R-squared: %.4f" % (1- mse/y_variance))
        
        return(mse)

    def evaluate_quantiles(self, chart=False, verbose=False):
    
        self.P_quantiles = np.zeros_like(self.P)
        self.Y_quantiles = np.zeros_like(self.Y)
        
        # compute score for predicted quantiles vs. actual (expected) quantiles
        N_QUANTILES=5
        for row in range(self.startindex, self.P_quantiles.shape[0]):
            #print(self.P[row])
            self.P_quantiles[row] = pd.qcut(self.P[row], N_QUANTILES, range(N_QUANTILES))
            self.Y_quantiles[row] = pd.qcut(self.Y[row], N_QUANTILES, range(N_QUANTILES))

        pred_quantiles = self.P_quantiles[self.startindex:]
        true_quantiles = self.Y_quantiles[self.startindex:]

        nrows, ncols = pred_quantiles.shape

        pred_quantiles = pred_quantiles.reshape(nrows*ncols)
        true_quantiles = true_quantiles.reshape(nrows*ncols) 
        
        nrows = nrows * ncols
        conf_mat = confusion_matrix(pred_quantiles, true_quantiles)
        if chart:
            fig, ax = plt.subplots(figsize=(10,10))
            sns.heatmap(conf_mat, annot=True, fmt='d')
            plt.ylabel('Actual')
            plt.xlabel('Predicted')
            plt.show()
            
        if verbose:
            print(chisquare(conf_mat))

        conf_mat_expected = np.array([[0.64, 0.16],[0.16, 0.04]])

        myscores = []
        for q in range(5):
            temp_pred = pred_quantiles == q
            temp_actual = true_quantiles == q
            conf_mat = confusion_matrix(temp_pred, temp_actual) / float(nrows)
            diff_mat = conf_mat - conf_mat_expected
            if verbose:
                print(conf_mat)
                print(diff_mat)
                print(chisquare(conf_mat.reshape(4)*nrows, conf_mat_expected.reshape(4)*nrows))
            # probably no valid statistical interpretation but 
            # average of improvement in true positive % and true negative %
            myscore = (diff_mat[0][0] + diff_mat[1][1])/2
            myscores.append(myscore)

        # average of my score for top and bottom quintiles
        finalscore = (myscores[0] + myscores[4])/2
        if verbose:
            print("Score: %f" % (finalscore))
            
        return finalscore

    def walkforward_xval (self, n_splits=5, verbose=False):
        """quick and dirty genreturns, with a step"""
        # generate k-folds
        kf = KFold(n_splits=n_splits)
        kf.get_n_splits(X)
        last_indexes = []
        for train_index, test_index in kf.split(X):
            # use test_index as last index to train
            last_index = test_index[-1] + 1
            last_indexes.append(last_index)
        print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))
        #override startindex
        self.startindex = last_indexes[0]
        return self.gen_predictions(splits=last_indexes, verbose=verbose)
    
    def gen_returns(self, port_returns_func, verbose=False):

        self.R = np.zeros(self.P.shape[0])
        first_pred_month=self.startindex
        
        indcount = [0] * self.ycols
        longcount = [0] * self.ycols
        shortcount = [0] * self.ycols
        
        for month_index in range(first_pred_month, nrows-1):
            return_month = month_index + 1
            port_return, long_indexes, short_indexes = port_returns_func(self.P[month_index], 
                                                                         self.X[return_month])
            self.R[return_month] = port_return
            
            for i in long_indexes:
                indcount[i] += 1
                longcount[i] += 1
            for i in short_indexes:
                indcount[i] += 1
                shortcount[i] += 1
                
        for i in range(len(responses)):
            print("%s: long %d times, short %d times, total %d times" % (predictors[i], 
                                                                         longcount[i], 
                                                                         shortcount[i], 
                                                                         indcount[i]))
        return self.R

    def report_returns(self, start_date='01/01/1970', freq='M'):

        first_pred_month=self.startindex        
        results = self.R[first_pred_month:]
        index = pd.date_range(start_date,periods=results.shape[0], freq=freq)
        perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
        perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)
        self.cumulative_return = perfdata['Equity']

        stats = perfdata['Equity'].calc_stats()
        
        retframe = pd.DataFrame([stats.stats.loc['start'],
                                 stats.stats.loc['end'],
                                 stats.stats.loc['cagr'],
                                 stats.stats.loc['yearly_vol'],
                                 stats.stats.loc['yearly_sharpe'],
                                 stats.stats.loc['max_drawdown'],
                                 ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                                ],
                                index = ['start',
                                         'end',
                                         'cagr',
                                         'yearly_vol',
                                         'yearly_sharpe',
                                         'max_drawdown',
                                         'sortino',
                                        ],
                                columns=['Value'])   
        return retframe

NameError: name 'FIRST_TRAIN_MONTHS' is not defined

In [12]:
NUMSTOCKS = 6 # top quintile (and bottom)

def calc_returns(prediction_row, return_row, numstocks=NUMSTOCKS, verbose=False):

    # ensure nan sorts to top for shorts
    short_sort_array = [999999 if np.isnan(x) else x for x in prediction_row]
    # pick bottom numstocks
    select_array = np.argsort(short_sort_array)
    short_indexes = select_array[:numstocks]

    # ensure nan sorts to bottom for longs
    long_sort_array = [-999999 if np.isnan(x) else x for x in prediction_row]
    # pick top numstocks
    select_array = np.argsort(long_sort_array)
    long_indexes = select_array[-numstocks:]
    
    if verbose:
        print("Longs: %s" %(str([(i,prediction_row[i]) for i in long_indexes])))
        print("Shorts: %s" %(str([(i,prediction_row[i]) for i in short_indexes])))

    # compute equal weighted long/short return
    return np.mean(return_row[long_indexes])/2 - np.mean(return_row[short_indexes])/2, long_indexes, short_indexes


In [13]:
start_date_int = data.index[FIRST_TRAIN_MONTHS]
start_year, start_month = start_date_int // 100, start_date_int % 100
start_date_str = "%02d/%02d/%d" % (start_month, 1, start_year)
start_date_str

'01/01/1970'

In [15]:
# use keras instead of sklearn MLPRegressor
# wrap keras model in a class 
# multioutput 30 predictions simultaneously to speed up 
# fit takes a list of response ys, predict returns a list of y_predict arrays
# no coef_dict
INPUT_DIM = X.shape[1]
print(INPUT_DIM)
OUTPUT_DIM = len(responses) # 30
BATCH_SIZE = 32
EPOCHS=500
# 500 is arbitrary, which bugs me
# something one could do is backtest and walk-forward xval with an xval set
# leave 30 months out of training fold, train until error on those 30 months is minimized, eval on following fold
# the other thing that bugs me is higher epochs take longer and yield worse results, there's some 'regularization thruogh under-training' going on
# but stochastically will never train at same speed, so you can pick a model and it might need more or less training than 100 in production

class KerasBacktestModel(object):

    def __init__(self, 
                 n_hidden_layers = 2,
                 hidden_layer_size = 32,
                 reg_penalty = 0.0001,
                 epochs=EPOCHS,
                 verbose=True):
        
        self.epochs=epochs
        
        main_input = Input(shape=(INPUT_DIM,),
                           dtype='float32', 
                           name='main_input')
        lastlayer=main_input
        
        for i in range(n_hidden_layers):
            if verbose:
                print("layer %d size %d, reg_penalty %.8f" % (i + 1, 
                                                              hidden_layer_size, 
                                                              reg_penalty, 
                                                             ))
            lastlayer = Dense(units = hidden_layer_size, 
                              activation = 'relu',
                              kernel_initializer = keras.initializers.glorot_uniform(),
                              kernel_regularizer=keras.regularizers.l1(reg_penalty),
                              name = "Dense%02d" % i)(lastlayer)
            
        outputs = []
        for i in range(OUTPUT_DIM):
            # OUTPUT_DIM outputs
            outputs.append(Dense(1,
                                 activation='linear',
                                 name='output%02d' % i)(lastlayer)
                          )
            
        self.model = Model(inputs=[main_input], outputs=outputs)
        if verbose:
            print(self.model.summary())
            
        self.model.compile(loss="mse", optimizer="adam", loss_weights=[1.]*OUTPUT_DIM)
        
    def fit(self, X, Y, epochs=None, verbose=False):
        # convert Y to list of ys
        Y_list = [Y[:,i] for i in range(OUTPUT_DIM)]
        
        if epochs:
            self.epochs = epochs
            
        return self.model.fit(X,
                              Y_list,
                              batch_size=BATCH_SIZE,
                              epochs=self.epochs,
                              verbose=verbose)
    
    def predict(self, X, verbose=False):
        y_list = self.model.predict(X)
        # convert list of ys to Y array
        npreds=len(y_list[0])
        Y_pred = [y.reshape(npreds) for y in y_list]
        Y_pred = np.array(Y_pred).transpose()    
        return Y_pred
    
    def save(self, modelname):
        self.model.save("%s.h5" % modelname)
        self.model.save_weights("%s_weights.h5" % modelname)


30


In [41]:
# no hidden layer, linear activation = LinearRegression
keras_model = KerasBacktestModel(n_hidden_layers = 0,
                                 reg_penalty = 0.0,
                                 verbose=True,
                                 epochs=2000)

backtestmodel = BacktestModel(X, Y, 
                              model=keras_model, 
                              coef_dict_param=coef_dict_all, 
                              startindex=FIRST_TRAIN_MONTHS)
print(backtestmodel.fit_predict(121, npredict=3, verbose=True))



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 30)           0                                            
__________________________________________________________________________________________________
output00 (Dense)                (None, 1)            31          main_input[0][0]                 
__________________________________________________________________________________________________
output01 (Dense)                (None, 1)            31          main_input[0][0]                 
__________________________________________________________________________________________________
output02 (Dense)                (None, 1)            31          main_input[0][0]                 
__________________________________________________________________________________________________
output03 (

121/121 [==============================] - 0s 3ms/step - loss: 1401.6747 - output00_loss: 39.9413 - output01_loss: 27.5050 - output02_loss: 42.4601 - output03_loss: 107.9682 - output04_loss: 39.8668 - output05_loss: 32.6809 - output06_loss: 36.5817 - output07_loss: 43.4710 - output08_loss: 73.3934 - output09_loss: 33.5015 - output10_loss: 40.3934 - output11_loss: 43.4002 - output12_loss: 22.9869 - output13_loss: 43.1747 - output14_loss: 30.9591 - output15_loss: 47.7792 - output16_loss: 33.9746 - output17_loss: 52.2127 - output18_loss: 34.9946 - output19_loss: 43.9257 - output20_loss: 25.6806 - output21_loss: 55.3574 - output22_loss: 64.6345 - output23_loss: 34.3240 - output24_loss: 68.1945 - output25_loss: 54.8904 - output26_loss: 28.7194 - output27_loss: 53.6113 - output28_loss: 45.8592 - output29_loss: 99.2322
Epoch 12/2000
121/121 [==============================] - 0s 4ms/step - loss: 1364.5824 - output00_loss: 39.4990 - output01_loss: 27.3662 - output02_loss: 42.1702 - output03_los

121/121 [==============================] - 1s 5ms/step - loss: 1106.4123 - output00_loss: 34.1079 - output01_loss: 24.8107 - output02_loss: 37.8847 - output03_loss: 62.6937 - output04_loss: 35.8649 - output05_loss: 30.2785 - output06_loss: 33.1454 - output07_loss: 34.2186 - output08_loss: 30.8580 - output09_loss: 31.1886 - output10_loss: 35.7552 - output11_loss: 38.9351 - output12_loss: 21.6501 - output13_loss: 39.9231 - output14_loss: 28.1346 - output15_loss: 42.5735 - output16_loss: 31.5250 - output17_loss: 42.1235 - output18_loss: 30.0015 - output19_loss: 36.6343 - output20_loss: 21.5348 - output21_loss: 50.4084 - output22_loss: 41.7209 - output23_loss: 30.2877 - output24_loss: 41.6171 - output25_loss: 50.6079 - output26_loss: 25.4863 - output27_loss: 49.3901 - output28_loss: 39.7716 - output29_loss: 53.2806
Epoch 30/2000
121/121 [==============================] - 1s 6ms/step - loss: 1099.7102 - output00_loss: 33.8422 - output01_loss: 24.6706 - output02_loss: 37.6679 - output03_loss

121/121 [==============================] - 1s 5ms/step - loss: 1019.5402 - output00_loss: 29.8865 - output01_loss: 22.7049 - output02_loss: 34.4882 - output03_loss: 58.2483 - output04_loss: 33.0396 - output05_loss: 28.3019 - output06_loss: 30.8246 - output07_loss: 32.8058 - output08_loss: 27.3864 - output09_loss: 29.4122 - output10_loss: 33.4096 - output11_loss: 36.5868 - output12_loss: 20.5807 - output13_loss: 37.4168 - output14_loss: 26.0860 - output15_loss: 38.9877 - output16_loss: 29.7508 - output17_loss: 39.8471 - output18_loss: 26.4746 - output19_loss: 31.5989 - output20_loss: 18.8072 - output21_loss: 46.8704 - output22_loss: 38.7093 - output23_loss: 27.2501 - output24_loss: 39.4350 - output25_loss: 47.0826 - output26_loss: 24.3356 - output27_loss: 45.8863 - output28_loss: 35.3379 - output29_loss: 47.9881
Epoch 47/2000
121/121 [==============================] - 0s 3ms/step - loss: 1015.4313 - output00_loss: 29.6738 - output01_loss: 22.5825 - output02_loss: 34.3419 - output03_loss

121/121 [==============================] - 1s 5ms/step - loss: 958.6062 - output00_loss: 26.7608 - output01_loss: 21.0658 - output02_loss: 31.9726 - output03_loss: 56.8055 - output04_loss: 31.1353 - output05_loss: 26.7286 - output06_loss: 29.0063 - output07_loss: 31.4877 - output08_loss: 26.5979 - output09_loss: 28.0719 - output10_loss: 31.4002 - output11_loss: 34.5859 - output12_loss: 19.7976 - output13_loss: 35.4348 - output14_loss: 24.5504 - output15_loss: 36.4803 - output16_loss: 28.3858 - output17_loss: 37.8589 - output18_loss: 23.8699 - output19_loss: 27.4845 - output20_loss: 17.0348 - output21_loss: 44.1553 - output22_loss: 36.3513 - output23_loss: 25.0079 - output24_loss: 38.0182 - output25_loss: 44.3133 - output26_loss: 23.2590 - output27_loss: 43.0278 - output28_loss: 32.1863 - output29_loss: 45.7715
Epoch 63/2000
121/121 [==============================] - 0s 4ms/step - loss: 955.1656 - output00_loss: 26.5796 - output01_loss: 20.9808 - output02_loss: 31.8220 - output03_loss: 

...

121/121 [==============================] - 1s 5ms/step - loss: 594.4618 - output00_loss: 12.3924 - output01_loss: 12.4962 - output02_loss: 20.7495 - output03_loss: 32.0463 - output04_loss: 23.9909 - output05_loss: 18.3921 - output06_loss: 18.1012 - output07_loss: 20.7254 - output08_loss: 13.9700 - output09_loss: 19.0920 - output10_loss: 19.0691 - output11_loss: 20.7766 - output12_loss: 16.5078 - output13_loss: 24.5116 - output14_loss: 17.0244 - output15_loss: 26.5181 - output16_loss: 20.7457 - output17_loss: 21.1267 - output18_loss: 14.1375 - output19_loss: 9.3525 - output20_loss: 11.0505 - output21_loss: 29.2454 - output22_loss: 22.2336 - output23_loss: 15.3992 - output24_loss: 21.2580 - output25_loss: 27.7700 - output26_loss: 14.5482 - output27_loss: 28.3429 - output28_loss: 18.9624 - output29_loss: 23.9256
Epoch 1917/2000
121/121 [==============================] - 1s 5ms/step - loss: 594.7780 - output00_loss: 12.4149 - output01_loss: 12.5048 - output02_loss: 20.7363 - output03_loss:

121/121 [==============================] - 1s 4ms/step - loss: 594.6100 - output00_loss: 12.3970 - output01_loss: 12.5005 - output02_loss: 20.7189 - output03_loss: 32.0729 - output04_loss: 23.9958 - output05_loss: 18.3980 - output06_loss: 18.1042 - output07_loss: 20.7314 - output08_loss: 13.9730 - output09_loss: 19.0914 - output10_loss: 19.0820 - output11_loss: 20.7694 - output12_loss: 16.5140 - output13_loss: 24.5258 - output14_loss: 17.0245 - output15_loss: 26.5281 - output16_loss: 20.7457 - output17_loss: 21.1182 - output18_loss: 14.1416 - output19_loss: 9.3623 - output20_loss: 11.0465 - output21_loss: 29.2611 - output22_loss: 22.2425 - output23_loss: 15.4007 - output24_loss: 21.2615 - output25_loss: 27.7827 - output26_loss: 14.5574 - output27_loss: 28.3537 - output28_loss: 18.9690 - output29_loss: 23.9398
Epoch 1935/2000
121/121 [==============================] - 1s 5ms/step - loss: 594.5998 - output00_loss: 12.3981 - output01_loss: 12.5005 - output02_loss: 20.7228 - output03_loss:

121/121 [==============================] - 1s 6ms/step - loss: 594.5431 - output00_loss: 12.3933 - output01_loss: 12.4986 - output02_loss: 20.7207 - output03_loss: 32.0419 - output04_loss: 24.0136 - output05_loss: 18.3923 - output06_loss: 18.1046 - output07_loss: 20.7233 - output08_loss: 13.9723 - output09_loss: 19.0893 - output10_loss: 19.0716 - output11_loss: 20.7678 - output12_loss: 16.5089 - output13_loss: 24.5281 - output14_loss: 17.0334 - output15_loss: 26.5325 - output16_loss: 20.7510 - output17_loss: 21.1236 - output18_loss: 14.1499 - output19_loss: 9.3519 - output20_loss: 11.0464 - output21_loss: 29.2500 - output22_loss: 22.2418 - output23_loss: 15.3930 - output24_loss: 21.2732 - output25_loss: 27.7761 - output26_loss: 14.5498 - output27_loss: 28.3449 - output28_loss: 18.9646 - output29_loss: 23.9345
Epoch 1953/2000
121/121 [==============================] - 0s 4ms/step - loss: 594.6965 - output00_loss: 12.3981 - output01_loss: 12.5043 - output02_loss: 20.7249 - output03_loss:

121/121 [==============================] - 1s 5ms/step - loss: 594.4610 - output00_loss: 12.3935 - output01_loss: 12.4962 - output02_loss: 20.7207 - output03_loss: 32.0538 - output04_loss: 23.9894 - output05_loss: 18.3910 - output06_loss: 18.1007 - output07_loss: 20.7278 - output08_loss: 13.9715 - output09_loss: 19.0876 - output10_loss: 19.0686 - output11_loss: 20.7646 - output12_loss: 16.5081 - output13_loss: 24.5167 - output14_loss: 17.0289 - output15_loss: 26.5229 - output16_loss: 20.7550 - output17_loss: 21.1203 - output18_loss: 14.1386 - output19_loss: 9.3523 - output20_loss: 11.0481 - output21_loss: 29.2494 - output22_loss: 22.2339 - output23_loss: 15.3991 - output24_loss: 21.2621 - output25_loss: 27.7747 - output26_loss: 14.5485 - output27_loss: 28.3418 - output28_loss: 18.9701 - output29_loss: 23.9253
Epoch 1971/2000
121/121 [==============================] - 0s 3ms/step - loss: 594.6144 - output00_loss: 12.3960 - output01_loss: 12.4991 - output02_loss: 20.7386 - output03_loss:

121/121 [==============================] - 0s 4ms/step - loss: 594.5335 - output00_loss: 12.3956 - output01_loss: 12.4933 - output02_loss: 20.7231 - output03_loss: 32.0379 - output04_loss: 24.0034 - output05_loss: 18.3896 - output06_loss: 18.1031 - output07_loss: 20.7261 - output08_loss: 13.9737 - output09_loss: 19.0864 - output10_loss: 19.0728 - output11_loss: 20.7675 - output12_loss: 16.5113 - output13_loss: 24.5127 - output14_loss: 17.0455 - output15_loss: 26.5245 - output16_loss: 20.7532 - output17_loss: 21.1279 - output18_loss: 14.1452 - output19_loss: 9.3599 - output20_loss: 11.0563 - output21_loss: 29.2541 - output22_loss: 22.2294 - output23_loss: 15.3999 - output24_loss: 21.2606 - output25_loss: 27.7728 - output26_loss: 14.5723 - output27_loss: 28.3441 - output28_loss: 18.9654 - output29_loss: 23.9260
Epoch 1987/2000
121/121 [==============================] - 0s 4ms/step - loss: 594.4582 - output00_loss: 12.3957 - output01_loss: 12.4947 - output02_loss: 20.7175 - output03_loss:

In [42]:
# pretty close to what was found using LR in previous notebook
nn_array = np.array([-4.733711,-5.3550034,-3.6393228,-12.371351,-6.451857,-3.0385206,-6.7478585,-2.3304634,-5.8298798,-6.57771,-5.7783785,-7.6337743,-6.595377,-4.276085,-5.0506864,-5.945791,-4.738456,-1.887593,-3.502194,-2.3597922,-3.7378693,-2.9263158,-5.225221,-6.38983,-6.79949,-10.010931,-4.6692963,-8.7250805,-5.444964,-6.462293,])
lr_array = np.array([-4.70219919,-5.3073358, -3.72029191, -12.42448745,-6.62775756, -2.90316148,-6.79960146,-2.23172984,-5.75101181,-6.66276641, -5.75265368,-7.71979359,-6.57864566,-4.32124795,-5.0355927, -5.92230241,-4.76675177,-1.96315925,-3.43369206,-2.30343164, -3.72794445,-2.91917105,-5.27753906,-6.40474001,-6.85223288,-10.16508667,-4.69016977,-8.8042513, -5.4481542, -6.40122815])
nn_array / lr_array -1

array([ 0.0067015 ,  0.00898146, -0.02176418, -0.00427675, -0.02653998,
        0.04662473, -0.0076097 ,  0.04424082,  0.01371376, -0.01276593,
        0.00447182, -0.01114269,  0.00254328, -0.01045137,  0.0029974 ,
        0.00396612, -0.00593607, -0.03849217,  0.01994994,  0.02446808,
        0.00266228,  0.00244753, -0.00991334, -0.00232796, -0.00769718,
       -0.01516521, -0.00445047, -0.00899234, -0.00058556,  0.00953955])

In [16]:
# run linear model,compare results
def create_keras_model(n_hidden_layers, layer_size, reg_penalty, verbose=False):
    return KerasBacktestModel(n_hidden_layers = n_hidden_layers,
                              hidden_layer_size = layer_size,
                              reg_penalty = reg_penalty,
                              epochs=500,
                              verbose=verbose)


In [44]:
MODELPREFIX = "NN"

n_hiddens = [1, 2, 3]
layer_sizes = [1, 2, 4]
reg_penalties = [0.0, 0.01, 1]
hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))
# should really just use a list and convert to dataframe
experiments = {}
sharpes = {}
quantile_scores = {}

for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    print("%s n_hidden_layers = %d, hidden_layer_size = %d, reg_penalty = %.6f" % 
          (time.strftime("%H:%M:%S"), n_hidden_layers, layer_size, reg_penalty))
    
    experiment_model = BacktestModel(X, Y, 
                                     create_keras_model(n_hidden_layers,layer_size,reg_penalty), 
                                     coef_dict_param="all", 
                                     startindex=FIRST_TRAIN_MONTHS)
    experiment_model.walkforward_xval(n_splits=5)
    score = experiment_model.evaluate_predictions()
    experiments[key] = score
    quantile_score = experiment_model.evaluate_quantiles(chart=False, verbose=False)
    quantile_scores[key] = quantile_score

    experiment_model.gen_returns(calc_returns, verbose=False)
    retframe = experiment_model.report_returns(start_date=start_date_str, freq='M')
    sharpe = retframe.loc['yearly_sharpe']
    sharpes[key] = sharpe.values[0]
    
    print("%s MSE: %f" % (str(key), score))
    print("%s Quantile score: %f" % (str(key), quantile_score))
    print("%s Sharpe: %f" % (str(key), sharpe))



11:04:45 Running 27 experiments
11:04:45 Running experiment 1 of 27
11:04:45 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.000000
11:04:45 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 42.3022
Variance: 39.6015
R-squared: -0.0682
Food: long 40 times, short 0 times, total 40 times
Beer: long 13 times, short 0 times, total 13 times
Smoke: long 134 times, short 0 times, total 134 times
Games: long 404 times, short 94 times, total 498 times
Books: long 129 times, short 110 times, total 239 times
Hshld: long 104 times, short 0 times, total 104 times
Clths: long 204 times, short 43 times, total 247 times
Hlth: long 67 times, short 100 times, total 167 times
Chems: long 18 times, short 434 times, total 452 times
Txtls: long 246 times, short 28 times, total 274 times
Cnstr: long 0 times, short 123 times, total 123 times
Steel: long 0 times, short 482 times, total 482 times
FabPr: long 0 times, short 13 times, total 13 times
ElcEq: long 28 times, short

14:27:49 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 45.8996
Variance: 39.6015
R-squared: -0.1590
Food: long 265 times, short 72 times, total 337 times
Beer: long 233 times, short 54 times, total 287 times
Smoke: long 313 times, short 16 times, total 329 times
Games: long 162 times, short 69 times, total 231 times
Books: long 97 times, short 21 times, total 118 times
Hshld: long 87 times, short 110 times, total 197 times
Clths: long 87 times, short 20 times, total 107 times
Hlth: long 129 times, short 19 times, total 148 times
Chems: long 52 times, short 298 times, total 350 times
Txtls: long 63 times, short 0 times, total 63 times
Cnstr: long 0 times, short 72 times, total 72 times
Steel: long 0 times, short 360 times, total 360 times
FabPr: long 2 times, short 180 times, total 182 times
ElcEq: long 86 times, short 4 times, total 90 times
Autos: long 0 times, short 71 times, total 71 times
Carry: long 96 times, short 50 times, total 146 times
Mines: long

17:51:28 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 43.9098
Variance: 39.6015
R-squared: -0.1088
Food: long 231 times, short 3 times, total 234 times
Beer: long 229 times, short 15 times, total 244 times
Smoke: long 336 times, short 0 times, total 336 times
Games: long 86 times, short 0 times, total 86 times
Books: long 1 times, short 0 times, total 1 times
Hshld: long 174 times, short 11 times, total 185 times
Clths: long 41 times, short 0 times, total 41 times
Hlth: long 0 times, short 0 times, total 0 times
Chems: long 0 times, short 400 times, total 400 times
Txtls: long 218 times, short 34 times, total 252 times
Cnstr: long 33 times, short 281 times, total 314 times
Steel: long 0 times, short 507 times, total 507 times
FabPr: long 85 times, short 279 times, total 364 times
ElcEq: long 97 times, short 47 times, total 144 times
Autos: long 0 times, short 201 times, total 201 times
Carry: long 93 times, short 87 times, total 180 times
Mines: long 144 t

21:15:46 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 66.8768
Variance: 39.6015
R-squared: -0.6887
Food: long 131 times, short 61 times, total 192 times
Beer: long 78 times, short 76 times, total 154 times
Smoke: long 319 times, short 5 times, total 324 times
Games: long 229 times, short 227 times, total 456 times
Books: long 152 times, short 25 times, total 177 times
Hshld: long 110 times, short 29 times, total 139 times
Clths: long 67 times, short 67 times, total 134 times
Hlth: long 82 times, short 44 times, total 126 times
Chems: long 40 times, short 383 times, total 423 times
Txtls: long 14 times, short 0 times, total 14 times
Cnstr: long 18 times, short 133 times, total 151 times
Steel: long 1 times, short 410 times, total 411 times
FabPr: long 1 times, short 91 times, total 92 times
ElcEq: long 7 times, short 1 times, total 8 times
Autos: long 0 times, short 118 times, total 118 times
Carry: long 156 times, short 67 times, total 223 times
Mines: lon

00:41:13 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 39.5694
Variance: 39.6015
R-squared: 0.0008
Food: long 272 times, short 0 times, total 272 times
Beer: long 137 times, short 0 times, total 137 times
Smoke: long 546 times, short 0 times, total 546 times
Games: long 0 times, short 0 times, total 0 times
Books: long 0 times, short 0 times, total 0 times
Hshld: long 137 times, short 0 times, total 137 times
Clths: long 0 times, short 0 times, total 0 times
Hlth: long 0 times, short 0 times, total 0 times
Chems: long 0 times, short 546 times, total 546 times
Txtls: long 274 times, short 0 times, total 274 times
Cnstr: long 0 times, short 137 times, total 137 times
Steel: long 0 times, short 546 times, total 546 times
FabPr: long 0 times, short 137 times, total 137 times
ElcEq: long 135 times, short 0 times, total 135 times
Autos: long 0 times, short 272 times, total 272 times
Carry: long 0 times, short 137 times, total 137 times
Mines: long 274 times, shor

04:08:59 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 190.4828
Variance: 39.6015
R-squared: -3.8100
Food: long 200 times, short 116 times, total 316 times
Beer: long 64 times, short 66 times, total 130 times
Smoke: long 381 times, short 40 times, total 421 times
Games: long 191 times, short 198 times, total 389 times
Books: long 102 times, short 6 times, total 108 times
Hshld: long 99 times, short 67 times, total 166 times
Clths: long 109 times, short 83 times, total 192 times
Hlth: long 115 times, short 44 times, total 159 times
Chems: long 0 times, short 290 times, total 290 times
Txtls: long 137 times, short 79 times, total 216 times
Cnstr: long 7 times, short 141 times, total 148 times
Steel: long 6 times, short 407 times, total 413 times
FabPr: long 0 times, short 97 times, total 97 times
ElcEq: long 9 times, short 44 times, total 53 times
Autos: long 43 times, short 147 times, total 190 times
Carry: long 48 times, short 140 times, total 188 times
Min

In [45]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
 
lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "MSE"])
# one row didn't converge properly - messes up plotly scales
#for i in list(lossframe.loc[lossframe['loss']> 1000].index):
#    lossframe.at[i, 'loss'] = 100
lossframe.sort_values(['MSE'])

,n_hidden_layers,layer_size,reg_penalty,MSE
24,2,1,1.00,39.566295
22,2,1,0.00,39.567365
18,3,1,0.00,39.568893
15,3,1,1.00,39.569404
21,1,1,0.00,42.302223
20,1,1,1.00,43.160871
26,2,1,0.01,43.909834
14,1,1,0.01,44.112892
13,1,2,1.00,45.899574
23,2,2,1.00,46.772164


In [46]:
# list sharpes
flatlist = [list(l[0]) + [l[1]] for l in sharpes.items()]
 
sharpeframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "sharpe"])
# one row didn't converge properly - messes up plotly scales
#for i in list(lossframe.loc[lossframe['loss']> 1000].index):
#    lossframe.at[i, 'loss'] = 100
#list(l) +
sharpeframe.sort_values(['sharpe'])

,n_hidden_layers,layer_size,reg_penalty,sharpe
26,2,1,0.01,-0.163397
15,3,1,1.00,-0.146977
24,2,1,1.00,-0.121825
22,2,1,0.00,-0.114876
18,3,1,0.00,-0.114876
25,2,2,0.00,-0.070384
4,2,4,1.00,-0.034060
23,2,2,1.00,-0.031730
10,2,2,0.01,0.025068
21,1,1,0.00,0.078467


In [47]:
# list quantile scores
flatlist = [list(l[0]) + [l[1]] for l in quantile_scores.items()]
 
qframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "qscore"])
# one row didn't converge properly - messes up plotly scales
#for i in list(lossframe.loc[lossframe['loss']> 1000].index):
#    lossframe.at[i, 'loss'] = 100
#list(l) +
qframe.sort_values(['qscore'])

,n_hidden_layers,layer_size,reg_penalty,qscore
4,2,4,1.00,0.001149
26,2,1,0.01,0.001362
10,2,2,0.01,0.001484
24,2,1,1.00,0.001697
11,1,2,0.00,0.001758
15,3,1,1.00,0.001819
21,1,1,0.00,0.001849
23,2,2,1.00,0.001849
19,3,1,0.01,0.001880
18,3,1,0.00,0.001910


In [48]:
sharpes_array = sharpeframe.values[:,3]
print(sharpes_array.shape)
mses_array = lossframe.values[:,3]
print(mses_array.shape)
qscore_array = qframe.values[:,3]
print(qscore_array.shape)

(27,)
(27,)
(27,)


In [49]:
def myscatter(arg1, arg2, names=None):
    
    plotdata = []
    
    plotdata.append(Scatter(
        x = arg1,
        y = arg2,
        mode = 'markers'
    ))
    
    layout = Layout(
        autosize=False,
        width=600,
        height=480,
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    
    fig = Figure(data=plotdata, layout=layout)
    
    return iplot(fig)
    
myscatter(sharpes_array, mses_array)
# MSEs worse than linear regression
# no very good Sharpes
# weak correlation between MSEs, Sharpes, which is perplexing

In [50]:
myscatter(sharpes_array, qscore_array)


In [51]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['MSE'].mean())


,MSE
n_hidden_layers,
1,48.181159
2,50.718667
3,69.926619


In [52]:
pd.DataFrame(lossframe.groupby(['layer_size'])['MSE'].mean())


,MSE
layer_size,
1,42.240978
2,54.930036
4,71.655431


In [53]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['MSE'].mean())


,MSE
reg_penalty,
0.00,49.621204
0.01,71.094595
1.00,48.110646


In [54]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[y_labels], columns=[x_labels], values=['MSE'])
#    print(pivot)
    # specify labels as strings, to force plotly to use a discrete axis
#    print(pivot.columns.levels[1]).values
#    print(lossframe[x_labels].dtype)
    
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
#    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" layers", y_suffix=" units")




In [55]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix=" layers", y_suffix=" p")


In [56]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix=" p", y_suffix="units")


In [ ]:
backtestmodel = BacktestModel(X, Y, 
                              model=create_keras_model(1,4,0.01), 
                              coef_dict_param="all", 
                              startindex=FIRST_TRAIN_MONTHS,
                              fit_missing='mean',
                              scaler=None)

backtestmodel.gen_predictions(verbose=False)
backtestmodel.evaluate_predictions()
backtestmodel.evaluate_quantiles(chart=True, verbose=True)
backtestmodel.gen_returns(calc_returns, verbose=False)
backtestmodel.report_returns(start_date=start_date_str, freq='M')


.........

In [ ]:
........
